In [17]:
import requests
import time
import datetime
from datetime import datetime, timedelta
import pandas as pd

In [18]:
mario = '59175380725'

In [19]:
def sendMessage(para, mensaje):
    url = 'http://localhost:3001/lead'
    
    data = {
        "message": mensaje,
        "phone": para
    }
    headers = {
        'Content-Type':'application/json'
    }
    print(data)
    response = requests.post(url, json=data, headers=headers)
    return response

In [20]:
def definir_trapiche(df):
    # extrae la hora
    df['solo_hora'] = df['hora'].str.split('T').str[1]
    # concatena fecha y hora
    df['fecha_hora'] = df['fecha'] + ' ' + df['solo_hora']
    # convierte la columna a tipo datetime
    df['fecha_hora'] = pd.to_datetime(df['fecha_hora'])
    # obtiene la fecha y hora actual
    hora_actual = datetime.now()
    # calcula una hora antes
    una_hora_antes = hora_actual - timedelta(hours=1)
    # filtra los resgistros de la ultima hora
    df_ultima_hora = df[(df['fecha_hora'] >= una_hora_antes) & (df['fecha_hora'] <= hora_actual)]
    trapiches = list(set(df_ultima_hora['trapiche']))
    if len(trapiches) == 0:
        return 0
    elif 1 in trapiches and 2 in trapiches:
        return 3
    elif 1 in trapiches:
        return 1
    elif 2 in trapiches:
        return 2
    print('=================== ERROR =====================')

In [21]:
def calcular_horas_espera(df_tcb):
    df = df_tcb.copy()
    # elimina todos los registro sin datos de fechaDocum y HoraDocum
    #df = df.dropna(subset=['canero'])
    df = df[df['dateDocum'] != '0000-00-00']
    
    #extrae la hora para FECHA DE INICIO
    df['horaDocum'] = df['horaDocum'].str.split('T').str[1]
    # concatena fecha y hora
    df['fecha_inicio'] = df['dateDocum'] + ' ' + df['horaDocum']
    # convierte la columna a tipo datetime
    df['fecha_inicio'] = pd.to_datetime(df['fecha_inicio'])

    #extraer la hora para FECHA DE FIN
    df['startTime'] = df['startTime'].str.split('T').str[1]
    # concatena fecha y hora
    df['fecha_fin'] = df['startDate'] + ' ' + df['startTime']
    # convierte la columna a tipo datetime
    df['fecha_fin'] = pd.to_datetime(df['fecha_fin'])

    #calcula la diferencia
    df['espera'] = (df['fecha_fin'] - df['fecha_inicio']).dt.total_seconds() / 3600

    #retorn la media
    return df['espera'].mean()

In [7]:
df_playa['dateCupo'] = pd.to_datetime(df_playa['dateCupo'])

NameError: name 'df_playa' is not defined

In [9]:
fecha_actual = pd.Timestamp('today').normalize()

In [10]:
df_actual = df_playa[(df_playa['dateCupo'] == fecha_actual) & (df_playa['status'] != 'SL')]

NameError: name 'df_playa' is not defined

In [22]:
def calcular_datos():
    df_playa = pd.read_excel(r'G:/Ingenio Azucarero Guabira S.A/COOR_GERENCIA_CANA - Parte_Horarios/ReportePlaya.xlsx')
    df_molienda = pd.read_excel(r'G:/Ingenio Azucarero Guabira S.A/COOR_GERENCIA_CANA - Parte_Horarios/Molienda.xlsx')
    df_trafCamBalanza = pd.read_excel(r'G:/Ingenio Azucarero Guabira S.A/COOR_GERENCIA_CANA - Parte_Horarios/TrafCamBalanza.xlsx')
    
    df_horarios = pd.read_excel(r'G:/Ingenio Azucarero Guabira S.A/COOR_GERENCIA_CANA - Parte_Horarios/Horarios.xlsx')
    
    df_res_molienda = pd.merge(df_molienda, df_horarios[['Hora', 'Orden_Hora']], left_on='hora2', right_on='Hora', how='left')
    
    #cantidad de paquetes
    #cantidad de caña disponible
    filtro = df_playa[(df_playa['status'] == 'PL') | (df_playa['status'] == 'IN')]
    cantidad_paquetes = filtro['cantPqt'].sum()
    cana_disponible = cantidad_paquetes * 14
    
    #promedio lleganda pq
    df_playa['dateCupo'] = pd.to_datetime(df_playa['dateCupo'])
    fecha_actual = pd.Timestamp('today').normalize()
    df_actual = df_playa[(df_playa['dateCupo'] == fecha_actual) & (df_playa['status'] != 'SL')]
    df_actual['Hora_Entera'] = df_actual['horaDocum'].str[11:13].astype(int)
    max_hora_ent = df_actual['Hora_Entera'].max() - 3
    filtered_df = df_actual[df_actual['Hora_Entera'] >= max_hora_ent]
    sum_cant_pqt = filtered_df['cantPqt'].sum()
    promedio_llegada_pq = sum_cant_pqt / 3
    
    #trapiches
    # trapiche1    210 tn/ha    15 paquetes
    # trapiche2    690 tn/ha    49 paquetes
    
    #horas molienda
    horas_molienda_t1 = cantidad_paquetes / 15
    horas_molienda_t2 = cantidad_paquetes / 49
    horas_molienda_total = cantidad_paquetes / (15 + 49)
    
    #total paquetes resto dia
    total_paquetes_resto_dia_t1 = promedio_llegada_pq * horas_molienda_t1
    total_paquetes_resto_dia_t2 = promedio_llegada_pq * horas_molienda_t2
    total_paquetes_resto_dia_total = promedio_llegada_pq * horas_molienda_total
    
    #toneladas
    toneladas = df_molienda['netWeight'].sum() / 1000
    
    #planificacion actual
    planificacion_actual_t1 = df_res_molienda['Orden_Hora'].max() * 210
    planificacion_actual_t2 = df_res_molienda['Orden_Hora'].max() * 690
    planificacion_actual_total = df_res_molienda['Orden_Hora'].max() * (210 + 690)
    
    #diferencia actual
    diferencia_actual_t1 = toneladas - planificacion_actual_t1
    diferencia_actual_t2 = toneladas - planificacion_actual_t2
    diferencia_actual_total = toneladas - planificacion_actual_total
    
    #orden hora
    orden_hora = 24 - df_res_molienda['Orden_Hora'].max()
    
    #toneladas promedio
    toneladas_prom = (df_molienda['netWeight'].sum() / 1000) / (24 - orden_hora)

    #total horas
    total_horas_t1 = total_paquetes_resto_dia_t1 / (15) + horas_molienda_total
    total_horas_t2 = total_paquetes_resto_dia_t2 / (49) + horas_molienda_total
    total_horas_total = total_paquetes_resto_dia_total / (15 + 49) + horas_molienda_total

    #molienda segun promedio
    molienda_s_promedio = (toneladas_prom * orden_hora) + toneladas

    #molienda segun estimado
    molienda_s_estimado_t1 = toneladas + orden_hora * 210
    molienda_s_estimado_t2 = toneladas + orden_hora * 690
    molienda_s_estimado_total = toneladas + orden_hora * (210 + 690)

    #tiempo espera
    espera =  calcular_horas_espera(df_trafCamBalanza)
    
    trapiches = definir_trapiche(df_molienda)
    
    if trapiches == 0:
        mensaje = f'''*REPORTE*
*⚙️ Trapiches:* Detenidos
*🚛 Paquetes disponibles:* {round(cantidad_paquetes,2)}
*🔢 Toneladas aprox.:* {round(cana_disponible,2)}
*⏱️ Promedio llegada pq.:* {round(promedio_llegada_pq,2)}
*📈 Paquetes estimados:* ---
*🕰️Total horas abas.:* ---
*⏳Tiempo espera:* {round(espera,2)}
*🎋 Molienda actual:* {round(toneladas,2)}
*📅 Planificacion actual:* ---
*🔻 Diferencia actual:* ---
*🕒 Promedio horario:* {round(toneladas_prom,2)}
*🏭Molienda segun promedio:* {round(molienda_s_promedio,2)}
*📊Molienda segun estimacion:* ---'''
    
    elif trapiches == 1:
        mensaje = f'''*REPORTE*
*⚙️ Trapiches:* solo 01
*🚛 Paquetes disponibles:* {round(cantidad_paquetes,2)}
*🔢 Toneladas aprox.:* {round(cana_disponible,2)}
*⏱️ Promedio llegada pq.:* {round(promedio_llegada_pq,2)}
*📈 Paquetes estimados:* {round(total_paquetes_resto_dia_t1,2)}
*🕰️Total horas abas.:* {round(total_horas_t1,2)}
*⏳Tiempo espera:* {round(espera,2)}
*🎋 Molienda actual:* {round(toneladas,2)}
*📅 Planificacion actual:* {round(planificacion_actual_t1,2)}
*🔻 Diferencia actual:* {round(diferencia_actual_t1,2)}
*🕒 Promedio horario:* {round(toneladas_prom,2)}
*🏭Molienda segun promedio:* {round(molienda_s_promedio,2)}
*📊Molienda segun estimacion:* {round(molienda_s_estimado_t1,2)}'''
    
    elif trapiches == 2:
        mensaje = f'''*REPORTE*
*⚙️ Trapiches:* solo 02
*🚛 Paquetes disponibles:* {round(cantidad_paquetes,2)}
*🔢 Toneladas aprox.:* {round(cana_disponible,2)}
*⏱️ Promedio llegada pq.:* {round(promedio_llegada_pq,2)}
*📈 Paquetes estimados:* {round(total_paquetes_resto_dia_t2,2)}
*🕰️Total horas abas.:* {round(total_horas_t2,2)}
*⏳Tiempo espera:* {round(espera,2)}
*🎋 Molienda actual:* {round(toneladas,2)}
*📅 Planificacion actual:* {round(planificacion_actual_t2,2)}
*🔻 Diferencia actual:* {round(diferencia_actual_t2,2)}
*🕒 Promedio horario:* {round(toneladas_prom,2)}
*🏭Molienda segun promedio:* {round(molienda_s_promedio,2)}
*📊Molienda segun estimacion:* {round(molienda_s_estimado_t2,2)}'''
    
    elif trapiches == 3:
        mensaje = f'''*REPORTE*
*⚙️ Trapiches:* 01 y 02
*🚛 Paquetes disponibles:* {round(cantidad_paquetes,2)}
*🔢 Toneladas aprox.:* {round(cana_disponible,2)}
*⏱️ Promedio llegada pq.:* {round(promedio_llegada_pq,2)}
*📈 Paquetes estimados:* {round(total_paquetes_resto_dia_total,2)}
*🕰️Total horas abas.:* {round(total_horas_total,2)}
*⏳Tiempo espera:* {round(espera,2)}
*🎋 Molienda actual:* {round(toneladas,2)}
*📅 Planificacion actual:* {round(planificacion_actual_total,2)}
*🔻 Diferencia actual:* {round(diferencia_actual_total,2)}
*🕒 Promedio horario:* {round(toneladas_prom,2)}
*🏭Molienda segun promedio:* {round(molienda_s_promedio,2)}
*📊Molienda segun estimacion:* {round(molienda_s_estimado_total,2)}'''
    
    return mensaje

In [23]:
m = calcular_datos()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_32572\891701084.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_actual['Hora_Entera'] = df_actual['horaDocum'].str[11:13].astype(int)


In [24]:
print(m)

*REPORTE*
*⚙️ Trapiches:* solo 02
*🚛 Paquetes disponibles:* 53
*🔢 Toneladas aprox.:* 742
*⏱️ Promedio llegada pq.:* 17.67
*📈 Paquetes estimados:* 19.11
*🕰️Total horas abas.:* 1.22
*⏳Tiempo espera:* 3.46
*🎋 Molienda actual:* 2706.24
*📅 Planificacion actual:* 2760
*🔻 Diferencia actual:* -53.76
*🕒 Promedio horario:* 676.56
*🏭Molienda segun promedio:* 16237.44
*📊Molienda segun estimacion:* 16506.24


In [ ]:
while True:
    msj = calcular_datos()
    sendMessage(mario, msj)
    now = datetime.now()
    print('Mensaje enviado: ' + str(now))
    time.sleep(3600)

{'message': '*REPORTE*\n*⚙️ Trapiches:* solo 01\n*🚛 Paquetes disponibles:* 40\n*🔢 Toneladas aprox.:* 560\n*⏱️ Promedio llegada pq.:* 45.67\n*📈 Paquetes estimados:* 121.78\n*🕰️Total horas abas.:* 8.74\n*⏳Tiempo espera:* 1.09\n*🎋 Molienda actual:* 1652.82\n*📅 Planificacion actual:* 1050\n*🔻 Diferencia actual:* 602.82\n*🕒 Promedio horario:* 330.56\n*🏭Molienda segun promedio:* 7933.54\n*📊Molienda segun estimacion:* 5642.82', 'phone': '59175380725'}
